In [ ]:
!pip install geopandas


In [ ]:
pip list

In [85]:
import pandas as pd
import arcpy
from arcpy import env
import geopandas as gpd
import os
import csv
import multiprocessing

In [ ]:
# Set the current workspace 
folder_path = "G:/usgrids2020/for_join_data/"
gdb_path = r"G:/usgrids2020/gdbs/"
gdb_list = os.listdir(gdb_path)
gdb_list.sort()
for state_gdb in gdb_list[2:]:
    env.workspace = os.path.join(gdb_path, state_gdb)
    env.overwriteOutput = True
    print(env.workspace)
    # feature class
    # Set the local parameters
    joinTable = os.path.join(folder_path, f"{state_gdb[3:5]}_inputs.csv")
    print(joinTable)
    # create in memory path
    memory = 'in_memory/'
    mem_table_path = os.path.join(memory, 'csv')

    #create the in memory table
    arcpy.CopyRows_management(joinTable, mem_table_path)  
    #Add join from inputs.csv to feature in gdb 
    joined_table = arcpy.management.AddJoin(
        in_layer_or_view=f"usa{state_gdb[3:5]}_admin5_boundaries",
        in_field="GEOID20",
        join_table=joinTable,
        join_field="GEOID20",
        join_type="KEEP_ALL",
        index_join_fields="NO_INDEX_JOIN_FIELDS",
        rebuild_index="NO_REBUILD_INDEX"
    )

    arcpy.management.CopyFeatures(joined_table, "joinedtable")
    print("Done!")
    # Join two feature classes by the zonecode field and only carry 
    # over the land use and land cover fields
    # arcpy.management.JoinField(fc, joinField, mem_table_path, joinField)

In [ ]:
# from arcpy import env
# env.overwriteOutput = True
# out_path = 'G:/usgrids2020'

# arcpy.management.CreateFileGDB(out_path,"processed_data.gdb")

# env.workspace = bounds
# in_rows = "G:/usgrids2020/processed_census_data/delaware_us_final.csv"
# arcpy.conversion.TableToTable(in_rows, state_gdb,"delaware")


In [96]:
# gdb_path = r"G:/usgrids2020/gdbs"
# gdb_list = os.listdir(gdb_path)
# gdb_list.sort()
# state_gdb = gdb_list[9]
# arcpy.env.workspace = os.path.join(gdb_path, state_gdb)
# arcpy.env.overwriteOutput = True
# fc = arcpy.ListFeatureClasses()

# # feature class
# fc = f"usa{state_gdb[3:5]}_admin5_boundaries"

# # printing all fields in fc
# fields = [f.name for f in arcpy.ListFields(fc)]

# csvFile = r"G:/usgrids2020/for_join_data/de_inputs.csv"

def csvToDict(csvFile):
    csvDict={}
    try:
        with open(csvFile,"r") as infile:
            reader=csv.reader(infile)
            i=0
            for row in reader: 
                # print(row)       
                if i==0:
                    # the first row is the header
                    fields=row
                    i+=1
                else:
                    csvDict[row[0]]=row
        return((fields,csvDict))
    except Exception as err:
        print(f"Unexpected {err=}, {type(err)=}")
        print(arcpy.GetMessages())
        return("ERROR")
    
def process(fc):
    name = os.path.basename(fc)
    stateCode = name[3:5]
    csvFile = f"G:/usgrids2020/for_join_data/{stateCode}_inputs.csv"
    arcpy.env.workspace = f"G:/usgrids2020/test_gdb/usa{stateCode}.gdb"
    if not arcpy.Exists(csvFile):
        return ((0,csvFile+"does not exist"))
    response = csvToDict(csvFile)
    csvFields = response[0]
    csvDict = response[1]
    if response=="ERROR":
        return ((0,"CSVDICT ERROR"))
    fcInMem = "in_memory/test"
    arcpy.management.CopyFeatures(fc, fcInMem)
    arcpy.conversion.TableToTable(csvFile, "in_memory", "tableInMem")
    arcpy.management.Merge([fc, "in_memory/tableInMem"], fcInMem)
    out_path = f"G:/usgrids2020/test_gdb/usa{stateCode}.gdb/{stateCode}_joined"
    with arcpy.da.UpdateCursor(fcInMem, csvFields) as cursor:
        for row in cursor:
            header = row[0]
            if header in csvDict:
                csvData = csvDict[header]
                cursor.updateRow(csvData)
    arcpy.management.CopyFeatures(fcInMem, out_path)

def main():
    readTime = datetime.datetime.now()
    gdb_path = r"G:/usgrids2020/test_gdb/"
    gdb_list = os.listdir(gdb_path)[:39]
    gdb_list.sort()
    procList = arcpy.ListWorkspaces("*")
    procList = [os.path.join(gdb_path, state, f"usa{state[3:5]}_admin5_boundaries") for state in gdb_list]
    for p in procList:
        print(f"Processing for {p}")
        print(process(p))
        print(f"Script Complete in {str(datetime.datetime.now()- readTime)}")
# this is for multicore processing
    multiprocessing.set_executable(os.path.join(get_install_path(), 'pythonw.exe'))
    pool = multiprocessing.Pool(processes=32, maxtasksperchild=1)# processes should = number of cores
    results = pool.map(process, procList)
    for result in results:
        print(result)        
    # Synchronize the main process with the job processes to
    # ensure proper cleanup.
    pool.close()
    pool.join()

if __name__ == "__main__":
    main()

Processing for G:/usgrids2020/test_gdb/usaak.gdb\usaak_admin5_boundaries
None
Script Complete in 0:00:24.335508
Processing for G:/usgrids2020/test_gdb/usaal.gdb\usaal_admin5_boundaries


KeyboardInterrupt: 

In [35]:
out_path = "G:/usgrids2020/gdbs/usadetest.gdb/test"
print(os.path.basename(fc)[3:5])
# with arcpy.da.UpdateCursor(fcInMem, csvFields) as cursor:
#     for row in cursor:
#         header = row[0]
#         if header in csvDict:
#             csvData = csvDict[header]
#             cursor.updateRow(csvData)
# arcpy.management.CopyFeatures(fcInMem, out_path)

de


In [51]:
test_gdb = arcpy.ListWorkspaces("*")
for test in test_gdb:
    print(test)

G:/usgrids2020/test_gdb\usaak.gdb
G:/usgrids2020/test_gdb\usaal.gdb
G:/usgrids2020/test_gdb\usaar.gdb
G:/usgrids2020/test_gdb\usaaz.gdb
G:/usgrids2020/test_gdb\usaca.gdb
G:/usgrids2020/test_gdb\usaco.gdb
G:/usgrids2020/test_gdb\usact.gdb
G:/usgrids2020/test_gdb\usadc.gdb
G:/usgrids2020/test_gdb\usade.gdb
G:/usgrids2020/test_gdb\usafl.gdb
G:/usgrids2020/test_gdb\usaga.gdb
G:/usgrids2020/test_gdb\usahi.gdb
G:/usgrids2020/test_gdb\usaia.gdb
G:/usgrids2020/test_gdb\usaid.gdb
G:/usgrids2020/test_gdb\usail.gdb
G:/usgrids2020/test_gdb\usain.gdb
G:/usgrids2020/test_gdb\usaks.gdb
G:/usgrids2020/test_gdb\usaky.gdb
G:/usgrids2020/test_gdb\usala.gdb
G:/usgrids2020/test_gdb\usama.gdb
G:/usgrids2020/test_gdb\usamd.gdb
G:/usgrids2020/test_gdb\usame.gdb
G:/usgrids2020/test_gdb\usami.gdb
G:/usgrids2020/test_gdb\usamn.gdb
G:/usgrids2020/test_gdb\usamo.gdb
G:/usgrids2020/test_gdb\usams.gdb
G:/usgrids2020/test_gdb\usamt.gdb
G:/usgrids2020/test_gdb\usanc.gdb
G:/usgrids2020/test_gdb\usand.gdb
G:/usgrids2020

In [ ]:
env.workspace